In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 13.3 MB/s 
     |████████████████████████████████| 168 kB 61.9 MB/s 
     |████████████████████████████████| 182 kB 76.5 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 77.5 MB/s 
     |████████████████████████████████| 166 kB 78.5 MB/s 
     |████████████████████████████████| 166 kB 82.1 MB/s 
     |████████████████████████████████| 162 kB 81.7 MB/s 
     |████████████████████████████████| 162 kB 77.1 MB/s 
     |████████████████████████████████| 158 kB 84.4 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 85.5 MB/s 
     |████████████████████████████████| 157 kB 85.0 MB/s 
     |████████████████████████████████| 157 kB 87.5 MB/s 
     |████████████████████████████████| 157 kB 85.0 MB/s 
     |████████████████████████████████| 157 kB 82.3 MB/s 
     |████████████████████████████████| 157 kB 83.5 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [14]:
%env WANDB_NOTEBOOK_NAME = "ResNet101"

env: WANDB_NOTEBOOK_NAME="ResNet101"


In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [6]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [11]:
tf.keras.backend.clear_session()
resnet101 = tf.keras.applications.ResNet101(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling='avg')
model_resnet101=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet101,
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet101.trainable=False
model_resnet101.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 2048)              42658176  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 42,668,421
Trainable params: 10,245
Non-trainable params: 42,658,176
_________________________________________________________________


In [12]:
model_resnet101.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'],
              jit_compile = True
              )

In [13]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet101",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [15]:
%%time
epochs=40
history = model_resnet101.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback(
    compute_flops = True,
    save_model = True
)]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.5601 - accuracy: 0.3480

wandb: Adding directory to artifact (/content/wandb/run-20221208_092850-2m5q1lbn/files/model-best)... Done. 1.2s


22/22 [==============================] - 319s 10s/step - loss: 1.5601 - accuracy: 0.3480 - val_loss: 1.4241 - val_accuracy: 0.5143
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.3646 - accuracy: 0.4801

wandb: Adding directory to artifact (/content/wandb/run-20221208_092850-2m5q1lbn/files/model-best)... Done. 1.3s


...

22/22 [==============================] - 48s 2s/step - loss: 0.9573 - accuracy: 0.6364 - val_loss: 0.9820 - val_accuracy: 0.6629
Epoch 36/40
22/22 [==============================] - 7s 277ms/step - loss: 0.9768 - accuracy: 0.6449 - val_loss: 0.9832 - val_accuracy: 0.6514
Epoch 37/40
22/22 [==============================] - ETA: 0s - loss: 0.9710 - accuracy: 0.6293

wandb: Adding directory to artifact (/content/wandb/run-20221208_092850-2m5q1lbn/files/model-best)... Done. 1.1s


22/22 [==============================] - 47s 2s/step - loss: 0.9710 - accuracy: 0.6293 - val_loss: 0.9781 - val_accuracy: 0.6343
Epoch 38/40
22/22 [==============================] - 6s 256ms/step - loss: 0.9540 - accuracy: 0.6463 - val_loss: 0.9805 - val_accuracy: 0.6514
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.9532 - accuracy: 0.6420

wandb: Adding directory to artifact (/content/wandb/run-20221208_092850-2m5q1lbn/files/model-best)... Done. 1.0s


22/22 [==============================] - 48s 2s/step - loss: 0.9532 - accuracy: 0.6420 - val_loss: 0.9705 - val_accuracy: 0.6457
Epoch 40/40
22/22 [==============================] - 6s 255ms/step - loss: 0.9631 - accuracy: 0.6463 - val_loss: 0.9743 - val_accuracy: 0.6457
CPU times: user 18min 17s, sys: 1min 12s, total: 19min 30s
Wall time: 33min 10s


In [16]:
wandb.alert(title="Run finished",
            text='ResNet101')

In [17]:
wandb.finish()

accuracy,▁▄▅▅▅▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇▇▇▇█▇██████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▁▃▂▁▂▃▂▂▂▂▁▄▃▁▃▂▄▄▂▄▅▅▆▇▆█▅█▆▆▇▆█▇▆▇▇▇
val_loss,█▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁
GFLOPs,7.57704
accuracy,0.64631
best_epoch,38
best_val_loss,0.97049
epoch,39
loss,0.9631


In [18]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
